#Serialize

In [ ]:
#use this code to serialize images ->generate pickle files
#import packages as needed
import matplotlib.pyplot as plt
import numpy as np
import cv2
import random
import pickle
from pathlib import Path

###################preprocessing the data##############

#data set directory
with_mask_dir = '/content/dataset/with_mask'
without_mask_dir = '/content/dataset/without_mask'

with_mask_data=[]
without_mask_data=[]

# load data set and convert each image to an array of numbers
for path in os.listdir(with_mask_dir):
    full_path = os.path.join(with_mask_dir, path)
    if (os.path.isfile(full_path) and ('jpg' in full_path)):
      image = cv2.imread(full_path)
      #resize all images to the same size
      image = cv2.resize(image,(128,128))
      #scale down data set to 0-1 range
      img = cv2.normalize(image, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
      with_mask_data.append([img,0])

for path in os.listdir(without_mask_dir):
    full_path = os.path.join(without_mask_dir, path)
    if (os.path.isfile(full_path) and ('jpg' in full_path)):
      image = cv2.imread(full_path)
      #resize all images to the same size
      image = cv2.resize(image,(128,128))
      #scale down data set to 0-1 range
      img = cv2.normalize(image, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
      without_mask_data.append([img,1])

training_data = with_mask_data + without_mask_data;
random.shuffle(training_data);
x=[];
y=[];
for img, label in training_data:
  x.append(img);
  y.append(label)

#save the train data, pickles files reusable
pickle_out = open("images.pickle", "wb");
pickle.dump(x, pickle_out)
pickle_out.close()
pickle_out = open("labels.pickle", "wb");
pickle.dump(y, pickle_out)
pickle_out.close()

# Train

In [ ]:
#use this code to train a model
#import packages as needed
from keras.models import Sequential, model_from_json
from keras.layers import *
from keras.utils.vis_utils import plot_model
import keras
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import cv2
import random
import pickle
from pathlib import Path

import os
import tensorboard
import datetime
###################preprocessing the data##############

#data set directory
with_mask_dir = '/content/dataset/with_mask'
without_mask_dir = '/content/dataset/without_mask'

with_mask_data=[]
without_mask_data=[]
x=[];
y=[];
# load data set and convert each image to an array of numbers
try:
  for path in os.listdir(with_mask_dir):
      full_path = os.path.join(with_mask_dir, path)
      if (os.path.isfile(full_path) and ('jpg' in full_path)):
        image = cv2.imread(full_path)
        #resize all images to the same size
        image = cv2.resize(image,(128,128))
        #scale down data set to 0-1 range
        img = cv2.normalize(image, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
        with_mask_data.append([img,0])

  for path in os.listdir(without_mask_dir):
      full_path = os.path.join(without_mask_dir, path)
      if (os.path.isfile(full_path) and ('jpg' in full_path)):
        image = cv2.imread(full_path)
        #resize all images to the same size
        image = cv2.resize(image,(128,128))
        #scale down data set to 0-1 range
        img = cv2.normalize(image, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
        without_mask_data.append([img,1])

  training_data = with_mask_data + without_mask_data;
  random.shuffle(training_data);

  for img, label in training_data:
    x.append(img);
    y.append(label)
except:
  #load pickle file, use this if you are not using original dataset
  pickle_in = open("images.pickle", "rb");
  x = pickle.load(pickle_in)
  pickle_in = open("labels.pickle", "rb");
  y = pickle.load(pickle_in)
####################training the model##########

#create a model
model = keras.models.Sequential()
#add layers
#first layer
model.add(Conv2D(128, (3,3), activation="relu", input_shape=(128,128,3)))
#middle layers
model.add(Conv2D(100, (3,3), activation="relu"))
# use maxpooling to decrease number of parameters to speed up process
model.add(MaxPooling2D())
model.add(Conv2D(256, (3,3), activation="relu"))
model.add(MaxPooling2D())
model.add(Conv2D(512, (3,3), activation="relu"))
model.add(MaxPooling2D())
model.add(Conv2D(256, (3,3), activation="relu"))
model.add(Conv2D(128, (3,3), activation="relu"))
model.add(MaxPooling2D())
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(64))
model.add(Dense(64))
#last layer
model.add(Dense(1,activation="sigmoid"))
#print a summary of the model
model.summary()
plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)
#compile the model
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
#train the model
x = np.array(x);
y = np.array(y);
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)
model.fit(x, y, batch_size=8, epochs=10, validation_split=0.2, callbacks=[tensorboard_callback])

#save the trained model for later use
model.save("detectmask.model")
#save model weights
#model.sample_weights("model_weights.h5")


#TensorBoard

In [ ]:
# use this code to analyze/optimize the model with tensorboard
# Load the TensorBoard notebook extension 
%load_ext tensorboard
%tensorboard --logdir logs

#Predict

In [ ]:
#use this code to predict mask wearing using the trained model
import numpy as np
import cv2
from keras.models import Sequential, model_from_json
from google.colab.patches import cv2_imshow

import imutils
import dlib
import cv2

class_labels =[
               "with_mask",#0
               "without_mask"#1
]

#load images to predict
full_path = '/content/'
list_of_images_to_test = []
images = []
for path in os.listdir(full_path):
  if (('jpeg' in path) or ('jpg' in path)):
    image = cv2.imread(path)
    image = cv2.resize(image,(128,128))
    img = cv2.normalize(image, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
    images.append(image)
    list_of_images_to_test.append(img)
list_of_images_to_test = np.array(list_of_images_to_test)
if (len(list_of_images_to_test)==1):
  list_of_images_to_test = np.expand_dims(img, axis=0)
# load the trained model
model = tf.keras.models.load_model("detectmask.model")
results = model.predict(list_of_images_to_test)
i = 0
for res in results:
  print (res)
  index = ((res > 0.5)+0).ravel() #prediction is in class_labels[index[0]]
  #use haarcascade to detect face
  face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
  img2 = cv2.normalize(images[i], None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)
  img2 = cv2.cvtColor(img2,cv2.COLOR_BGR2GRAY)
  faces = face_cascade.detectMultiScale(images[i]) #expects 1 channel image
  for (x,y,w,h) in faces:
    if index == 0:#with mask, green rectangle
      cv2.rectangle(images[i], (x,y), (x+w,y+h), (0,128,0), 2) #rectangle color and line width
      cv2.putText(images[i], 'with mask', (x, y-5), cv2.FONT_HERSHEY_PLAIN, 0.5, (0,128,0), 1)
    else:#without mask, red rectange
      cv2.rectangle(images[i], (x,y), (x+w,y+h), (0,0,255), 2) #rectangle color and line width
      cv2.putText(images[i], 'without mask', (x, y-5), cv2.FONT_HERSHEY_PLAIN, 0.5, (0,0,255), 1)

  print(class_labels[index[0]])
  #plt.imshow(image)
  image = cv2.resize(images[i],(256,256))
  cv2_imshow(np.array(images[i], dtype = np.uint8 ))
  i += 1

